# (day3) Section 1: 再帰型ニューラルネットワークの構造

本書は、「深層学習後編（day3）レポート」の、「Section 1: 再帰型ニューラルネットワークの構造」についてのものです。


## 1. 要点まとめ


<!-- - page. 13 - page. 54

- 1-1 RNN 全体像
  - 1-1-1 RNN とは
  - 1-1-2 時系列データ
  - 1-1-2 RNN について
    - 全体像
    - 数学的記述
    - 特徴
- 1-2 BPTT
  - 1-2-1 BPTT とは
  - 1-2-2 BPTT の数学的記述
    - 1
    - 2
    - 3
    - 4
  - 1-2-3 BPTT の全体像 -->


### 1-1 RNN 全体像


#### 1-1-1 RNN とは

RNN ( Recurrent Neural Network ) とは、時系列データに対応可能な NN 。


#### 1-1-2 時系列データ

- 時間的順序を一定間隔ごとに観測される。

- 相互に統計的依存関係が認められるようなデータの系列。


#### 1-1-2 RNN について

<!-- page. 18 の例では、出力層が 2 。
2 ノードの予測値について、時系列の変化となる $ t $ の要素が追加されたもの。

$$
\textbf{y}_n^{(t)} = \textbf{y}_1^{(t)}, \textbf{y}_2^{(t)}
$$ -->


##### 全体像

<!-- - unfold

- W ( in )

- W ( out ) -->

page. 19 の、図の左部分は、 NN が計算した結果 z も入力にして計算を繰り返すことを意味する。

右の部分は、その流れを $ z_1, z_2, \cdots, z_4 $ の順序で計算していくことを意味する。


RNN では 3 種類の重みを使う。

page. 20 の図の、以下である。

- $ \textbf{W}_{(in)} $<br>
  入力 $ x_i $ に対する重み。

- $ \textbf{W}_{(out)} $<br>
  中間層の計算結果 $ z_i $ から、
  出力層の結果 $ y_i $ を得るための重み。

- $ \textbf{W} $<br>
  中間層の計算結果 $ z_i $ を、
  次の時間的順序の中間層 $ z_{i+1} $ に適用する重み。


##### 数学的記述とコード

$ \textbf{u}^t $ は、 page. 20 の、
中間層 $ z_t $ の入力値である。
入力に対する重み $ \textbf{W}_{(in)} $ を適用する。

$$
\textbf{u}^t = \textbf{W}_{(in)} \textbf{x}^t
+
\textbf{W} \textbf{z}^{t-1}
+
\textbf{b}
$$

今、処理している時間的順序 $ t $ の、
1 つ前の $ t - 1 $ の、
$ \textbf{z}^{t-1} $ を使っている。<br>
そして、それに、重み $ \textbf{W} $ を適用している。


`3_1_simple_RNN.ipynb` の対応するコードは、以下。

```python
        u[:,t+1] = np.dot(X, W_in) + np.dot(z[:,t].reshape(1, -1), W)
```

以下のループ処理の中にある。

```python
    # 時系列ループ
    for t in range(binary_dim):
```

以降の数式に対応するコードも同様に、上記のループの中にある。


$ \textbf{z}^t $ は、中間層の計算結果。

$$
\textbf{z}^t
=
f(
  \textbf{W}_{(in)} \textbf{x}^t
  +
  \textbf{W} \textbf{z}^{t-1}
  +
  \textbf{b}
)
$$

活性化関数 $ f() $ を適用する。


対応するコードは、以下。

```python
        z[:,t+1] = functions.sigmoid(u[:,t+1])
```

活性化関数は、シグモイド関数。


$ \textbf{v}^t $ は、出力層の結果。
重み $ \textbf{W}_{(out)} $ を適用する。

$$
\textbf{v}^t
=
\textbf{W}_{(out)} \textbf{z}^t + \textbf{c}
$$

$ \textbf{c} $ は、バイアス。


重み $ \textbf{W}_{(out)} $ を適用して、
その時間的順序の出力である、 $ \textbf{y}^t $ を得る。

$$
\textbf{y}^t = g(
  \textbf{W}_{(out)} \textbf{z}^t + \textbf{c}
)
$$


$ \textbf{v}^t, \textbf{y}^t $ に対応するコードは、以下。

```python
        y[:,t] = functions.sigmoid(np.dot(z[:,t+1].reshape(1, -1), W_out))
```

ここでの活性化関数も、シグモイド関数。


##### 特徴

再帰構造。

page. 20 の図の右側の通り、
$ z_{t} $ は $ z_{t - 1} $ の状態を保持し、
遡ると、 $ z_{0} $ の状態も保持している。
再帰的に次の時間的順序の結果が計算される。


### 1-2 BPTT

BPTT は、 BackproPagation Through Time の略である。
RNN における、誤差逆伝播法によるパラメータ調整方法。


#### 1-2-2 BPTT の数学的記述


##### 数学的記述とコード 1: 重みとバイアスの微分

重み $ \textbf{W}_{(in)} $ についての微分は以下の通り。

$$
\frac{\partial{E}}{\partial{\textbf{W}_{(in)}}}
=
\frac{\partial{E}}{\partial{\textbf{u}^t}}
\begin{bmatrix}
  \frac{\partial{\textbf{u}^t}}{\partial{\textbf{W}_{(in)}}}
\end{bmatrix}^\textit{T}
=
\delta^t
[\textbf{x}^t]^\textit{T}
$$

$ \delta^t $ は、誤差関数の微分を表したもの。

$ \begin{bmatrix}\end{bmatrix}^\textit{T} $ は、転置行列ではなく、 RNN において時間的順序を遡って計算することを意味する。


$ \delta^t
[\textbf{x}^t]^\textit{T} $ に対応するコードは以下の通り。

```python
# 勾配更新
...
W_in_grad += np.dot(X.T, delta[:,t].reshape(1,-1))
```


重み $ \textbf{W}_{(out)} $ についての微分は以下の通り。

$$
\frac{\partial{E}}{\partial{\textbf{W}_{(out)}}}
=
\frac{\partial{E}}{\partial{\textbf{v}^t}}
\begin{bmatrix}
  \frac{\partial{\textbf{v}^t}}{\partial{\textbf{W}_{(out)}}}
\end{bmatrix}^\textit{T}
=
\delta^{out, t}
[\textbf{z}^t]^\textit{T}
$$


$ \delta^{out, t}
[\textbf{z}^t]^\textit{T} $ に対応するコードは以下の通り。

```python
# 勾配更新
...
W_out_grad += np.dot(z[:,t+1].reshape(-1,1), delta_out[:,t].reshape(-1,1))
```


重み $ \textbf{W} $ についての微分は以下の通り。

$$
\frac{\partial{E}}{\partial{\textbf{W}}}
=
\frac{\partial{E}}{\partial{\textbf{u}^t}}
\begin{bmatrix}
  \frac{\partial{\textbf{u}^t}}{\partial{\textbf{W}}}
\end{bmatrix}^\textit{T}
=
\delta^{t}
[\textbf{z}^{t-1}]^\textit{T}
$$


$ \delta^{t}
[\textbf{z}^{t-1}]^\textit{T} $ に対応するコードは以下の通り。

```python
# 勾配更新
...
W_grad += np.dot(z[:,t].reshape(-1,1), delta[:,t].reshape(1,-1))
```


バイアス $ \textbf{b} $ についての微分は以下の通り。

$$
\frac{\partial{E}}{\partial{\textbf{b}}}
=
\frac{\partial{E}}{\partial{\textbf{u}^t}}
\frac{\partial{\textbf{u}^t}}{\partial{\textbf{b}}}
=
\delta^{t}
$$

$$
\because
\frac{\partial{\textbf{u}^t}}{\partial{\textbf{b}}} = 1
$$

$$
\textbf{u}^t = \textbf{W}_{(in)} \textbf{x}^t + \textbf{W} \textbf{x}^{t-1} + \textbf{b}
$$

において、 $ \textbf{W}_{(in)}, \textbf{W} $ の 2 つに対してバイアスが考えられるが、
足し合わされるので、合わせて $ \textbf{b} $ としている。


バイアス $ \textbf{c} $ についての微分は以下の通り。

$$
\frac{\partial{E}}{\partial{\textbf{c}}}
=
\frac{\partial{E}}{\partial{\textbf{v}^t}}
\frac{\partial{\textbf{v}^t}}{\partial{\textbf{c}}}
=
\delta^{out, t}
$$

$$
\because
\frac{\partial{\textbf{v}^t}}{\partial{\textbf{c}}} = 1
$$


##### 数学的記述とコード 2: 順伝播


中間層の入力値である $ \textbf{u}^t $ の数式は以下の通り。 ( page. 21 再掲 )

$$
\textbf{u}^t = \textbf{W}_{(in)} \textbf{x}^t + \textbf{W} \textbf{z}^{t-1} + \textbf{b}
$$


中間層の計算結果 $ \textbf{z}^t $ の数式は、以下の通り。 ( page. 21 再掲 )

$$
\textbf{z}^t
=
f(
  \textbf{W}_{(in)} \textbf{x}^t
  +
  \textbf{W} \textbf{z}^{t-1}
  +
  \textbf{b}
)
$$


出力層の結果 $ \textbf{v}^t $ の数式は、以下の通り。 ( page. 22 再掲 )

$$
\textbf{v}^t
=
\textbf{W}_{(out)} \textbf{z}^t + \textbf{c}
$$


出力 $ \textbf{y}^t $ の数式は、以下の通り。 ( page. 22 再掲 )

$$
\textbf{y}^t = g(
  \textbf{W}_{(out)} \textbf{z}^t + \textbf{c}
)
$$


##### 数学的記述とコード 3: $ \delta^t $


$$
\frac{\partial{E}}{\partial{\textbf{u}^t}}
=
\frac{\partial{E}}{\partial{\textbf{v}^t}}
\frac{\partial{\textbf{v}^t}}{\partial{\textbf{u}^t}} \\
=
\frac{\partial{E}}{\partial{\textbf{v}^t}}
\frac{\partial
  \{
    \textbf{W}_{(out)} f(\textbf{u}^t) + \textbf{c}
  \}
}
{\partial{\textbf{u}^t}} \\
=
f'(\textbf{u}^t) \textbf{W}_{(out)}^{\textit{T}}
\delta^{out, t}
$$

$$
\because
\frac{\partial{E}}{\partial{\textbf{v}^t}} = \delta^{out, t}
$$

また、 $ \textbf{W}^\textit{T} $ は、時間的順序を遡って計算することを意味。<br>
よって、 $ f'(\textbf{u}^t) \textbf{W}_{(out)}^{\textit{T}} $ の部分は、<br>
$ \textbf{W}_{(out)} f(\textbf{u}^t) $ を微分したもの。

これを、 $ \delta^t $ とおく。

$$
f'(\textbf{u}^t) \textbf{W}_{(out)}^{\textit{T}}
\delta^{out, t}
=
\delta^t
$$


$ \delta^{t} $ に対応するコードは、以下の通り。
( 勾配更新の前の行 )

```python
delta[:,t] = (np.dot(delta[:,t+1].T, W.T) + np.dot(delta_out[:,t].T, W_out.T)) * functions.d_sigmoid(u[:,t+1])

# 勾配更新
...
```


時間的順序を 1 つ遡った場合の、 $ \delta^{t-1} $ は、

$$
\delta^{t-1} = \frac{\partial{E}}{\partial{\textbf{u}^{t-1}}}
=
\frac{\partial{E}}{\partial{\textbf{u}^t}}
\frac{\partial{\textbf{u}^t}}{\partial{\textbf{u}^{t-1}}}
$$

これは、

$$
\textbf{u}^t = \textbf{W}_{(in)} \textbf{x}^t + \textbf{W} \textbf{z}^{t-1} + \textbf{b} \\
=
\textbf{W}_{(in)} \textbf{x}^t + \textbf{W} f(\textbf{u}^{t-1}) + \textbf{b}
$$

として、 $ \textbf{u}^{t-1} $ で微分できるから、<br>
$ \textbf{z}^{t-1} $ を経由して、

$$
=
\delta^t
\{
  \frac{\partial{\textbf{u}^t}}{\partial{\textbf{z}^{t-1}}}
  \frac{\partial{\textbf{z}^{t-1}}}{\partial{\textbf{u}^{t-1}}}
\} \\
=
\delta^t
\{
  \textbf{W} f'(\textbf{u}^{t-1})
\}
$$


$$
\because
\frac{\partial{\textbf{u}^t}}{\partial{\textbf{z}^{t-1}}}
=
\frac{
  \partial
  \{
    \textbf{W}_{(in)} \textbf{x}^t + \textbf{W} \textbf{z}^{t-1} + \textbf{b}
  \}
}
{\partial{\textbf{z}^{t-1}}} \\
= \textbf{W} \\
$$

$$
\frac{\partial{\textbf{z}^{t-1}}}{\partial{\textbf{u}^{t-1}}}
=
\frac{\partial
  \{
    f(\textbf{u}^{t-1})
  \}
}
{\partial{\textbf{u}^{t-1}}}
=
f'(\textbf{u}^{t-1})
$$


さらに、時間的順序において z 回遡る場合、<br>
( z は、中間層の結果 $ \textbf{z} $ とは別 )

$$
\delta^{t-z-1}
=
\delta^{t-z}
\{
  \textbf{W} f'(\textbf{u}^{t-z-1})
\}
$$

t - z 時間と、 t - z - 1 時間との関係を表す。


##### 数学的記述とコード 4: パラメータの更新


入力層の重み $ \textbf{W}_{(in)} $ の更新は、以下の通り。

$$
\textbf{W}_{(in)}^{t+1}
=
\textbf{W}_{(in)}^{t}
-
\epsilon \frac{\partial{E}}{\partial{\textbf{W}_{(in)}}} \\
=
\textbf{W}_{(in)}^{t}
-
\epsilon \sum_{z=0}^{\textit{T}_t}
\delta^{t-z}
\begin{bmatrix}
\textbf{x}^{t-z}
\end{bmatrix}^\textit{T}
$$


出力層の重み $ \textbf{W}_{(out)} $ の更新は、以下の通り。

$$
\textbf{W}_{(out)}^{t+1}
=
\textbf{W}_{(out)}^{t}
-
\epsilon \frac{\partial{E}}{\partial{\textbf{W}_{(out)}}} \\
=
\textbf{W}_{(out)}^{t}
-
\epsilon
\delta^{out, t}
\begin{bmatrix}
\textbf{z}^{t}
\end{bmatrix}^\textit{T}
$$

出力層への重みでは、前の時間的順序へ遡って計算しないので、 $ \sum_{z=0}^{\textit{T}_t} $ の総和はしない。


中間層の重み $ \textbf{W} $ の更新は、以下の通り。

$$
\textbf{W}^{t+1}
=
\textbf{W}^{t}
-
\epsilon \frac{\partial{E}}{\partial{\textbf{W}}} \\
=
\textbf{W}_{(in)}^{t}
-
\epsilon \sum_{z=0}^{\textit{T}_t}
\delta^{t-z}
\begin{bmatrix}
\textbf{z}^{t-z-1}
\end{bmatrix}^\textit{T}
$$


重みの更新に対応するコードは、以下の通り。

```python
    # 勾配適用
    W_in -= learning_rate * W_in_grad
    W_out -= learning_rate * W_out_grad
    W -= learning_rate * W_grad
```


入力層、中間層のバイアスの更新は、以下の通り。

$$
\textbf{b}^{t+1}
=
\textbf{b}^{t}
-
\epsilon \frac{\partial{E}}{\partial{\textbf{b}}} \\
=
\textbf{b}^{t}
-
\epsilon \sum_{z=0}^{\textit{T}_t}
\delta^{t-z}
$$


$$
\textbf{c}^{t+1}
=
\textbf{c}^{t}
-
\epsilon \frac{\partial{E}}{\partial{\textbf{c}}} \\
=
\textbf{c}^{t}
-
\epsilon
\delta^{out, t}
$$

出力層でのバイアスも、前の時間的順序へ遡って計算しないので、 $ \sum_{z=0}^{\textit{T}_t} $ の総和はしない。


#### 1-2-3 BPTT の全体像

$$
E^\textit{t} = loss(\textbf{y}^t, \textbf{d}^t) \\
= loss(g(\textbf{W}_{(out)} \textbf{W} + \textbf{c}), \textbf{d}^t) \\
= loss(g(
  \textbf{W}_{(out)}
  f(
    \textbf{W}_{(in)} \textbf{x}^t + \textbf{W} \textbf{z}^{t-1} + \textbf{b}
  )
  +
  \textbf{c}
  ), \textbf{d}^t)
$$


ここで、

$$
  f(
    \textbf{W}_{(in)} \textbf{x}^t + \textbf{W} \textbf{z}^{t-1} + \textbf{b}
  )
$$

の、 $ f() $ の中の部分をさらに展開すると、


$$
\textbf{W}_{(in)} \textbf{x}^t + \textbf{W} \textbf{z}^{t-1} + \textbf{b}
$$

$$
\textbf{W}_{(in)} \textbf{x}^t + \textbf{W} f(u^{t-1}) + \textbf{b}
$$

$$
\textbf{W}_{(in)} \textbf{x}^t
+
\textbf{W}
  f(
    \textbf{W}_{(in)} \textbf{x}^{t-1}
    +
    \textbf{W} \textbf{z}^{t-2} + \textbf{b}
  )
+
\textbf{b}
$$



$ z^t, z^{t-1}, z^{t-2} $ と、過去の時間を数珠繋がり、マトリョーシカの人形の様に入れ子に計算されていることを表す。


## 2. 確認テスト

以降の "page. " は、講義資料のページの番号です。


### page. 11

公式より、

$$
\frac{
  入力画像サイズ + 2 \times パディング - フィルターサイズ
}{ストライド}
+ 1
$$

$$
=
\frac{
  5 + 2 \times 1 - 3
}{2}
+ 1
= 3
$$

よって、出力画像のサイズは、 3 x 3 。


### page. 23

残り 1 つの重みは、

- $ \textbf{W} $<br>
  中間層の計算結果 $ z_i $ を、
  次の時間的順序の中間層 $ z_{i+1} $ に適用する重み。


### page. 36

$$
\frac{dz}{dx}
=
\frac{dz}{dt}
\frac{dt}{dx} \\
=
2t \times 1
=
2(x + y)
$$


### page. 45

$$
\textbf{y}^t = g(
  \textbf{W}_{(out)} \textbf{z}^t + \textbf{c}
)
$$

より、今回の図では、中間層の部分が、 $ \textbf{s}_t $ であるから、

$$
\textbf{y}^1 = g(
  \textbf{W}_{(out)} \textbf{s}^1 + \textbf{c}
)
\tag{1}
$$


$$
\textbf{z}^t
=
f(
  \textbf{W}_{(in)} \textbf{x}^t
  +
  \textbf{W} \textbf{z}^{t-1}
  +
  \textbf{b}
)
$$

より、同様に、

$$
\textbf{s}^1
=
f(
  \textbf{W}_{(in)} \textbf{x}^1
  +
  \textbf{W} \textbf{s}^{1-1}
  +
  \textbf{b}
)
=
f(
  \textbf{W}_{(in)} \textbf{x}^1
  +
  \textbf{W} \textbf{s}^{0}
  +
  \textbf{b}
)
\tag{2}
$$


$ y_1 $ は、 (1) と (2) から成る。
